# Test the network from pretrained models

In [ ]:
%load_ext autoreload
%autoreload 2

import sys 
import os
import torch
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

## Path to load the models

In [ ]:
current_load_model_path = "../models/version2"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

In [ ]:
from bert import FinetunedBert
finetuned_bert = FinetunedBert(device=device)
tokenizer = finetuned_bert.load(path=current_load_model_path+"_bert")
finetuned_bert_model = finetuned_bert.model

In [ ]:
from utils import EmbedUserInput
EmbedWrapper = lambda X: EmbedUserInput(tokenizer, finetuned_bert_model, X, device=device)

In [ ]:
from model import PrototypicalNetwork
from solver import Solver

In [ ]:
n_way = 2
proto_net = PrototypicalNetwork()
solver = Solver(proto_net, n_way=n_way, device=device)
solver.load(path=current_load_model_path+"_proto_net.pth")

## Your support set

In [ ]:
support = ["You are beautiful.", "People with hair are weird."]
labels = torch.tensor([1, 0]) # 1 = offensive, 0 = not offensive
embedded_support = EmbedWrapper(support)

## Your input query

In [ ]:
query = ["you is cool"]

pred_labels = solver.predict(embedded_support, labels, EmbedWrapper(query))
outcome = "offensive" if pred_labels.item() == 1 else "not offensive"
print(f"predicted: {outcome}")
